In [1]:
!pip install langchain langchain-community faiss-cpu sentence-transformers

import sys
from pathlib import Path
import pandas as pd

In [2]:
!pip install langchain-openai
!pip install -U langchain-huggingface

In [3]:
%cd '/content/drive/MyDrive/Tenx program/week-6'

from scripts.rag import (
    load_vectorstore,
    retrieve_chunks,
    build_prompt,
    generate_openrouter,
)

/content/drive/MyDrive/Tenx program/week-6


In [4]:
!pip install -U openai


## https://openrouter.ai/deepseek/deepseek-r1:free/api

In [5]:
from openai import OpenAI

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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"
)

response = client.chat.completions.create(
    model="deepseek/deepseek-r1:free",
    messages=[{"role": "user", "content": "Hello!"}]
)

print(response.choices[0].message.content)


Hello! How can I assist you today? 😊


In [6]:

# vector store & embedding
VSTORE_DIR   = Path("/content/drive/MyDrive/Tenx program/week-6/vector_store")
EMBED_MODEL    = "sentence-transformers/all-MiniLM-L6-v2"

# OpenRouter / Deepseek free model
OPENROUTER_URL = "https://openrouter.ai/api/v1"
OPENROUTER_KEY = "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"
OPENROUTER_MODEL = "deepseek/deepseek-r1:free"

# Prompt template
PROMPT_TPL = (
    "You are a financial analyst assistant for CrediTrust.\n"
    "Use only the following excerpts from customer complaints to answer the question.\n"
    "If the context doesn't contain the answer, say you don't have enough information.\n\n"
    "Context:\n{context}\n\nQuestion: {question}\n\nAnswer:"
)

# Retrieval params
K = 5

# Example question
QUESTION = "Why are customers unhappy with Buy Now, Pay Later?"


In [7]:
# 1) Load FAISS
vs = load_vectorstore(VSTORE_DIR, EMBED_MODEL)

# 2) Retrieve top-k chunks
chunks = retrieve_chunks(vs, QUESTION, k=K)

print(f"Retrieved {len(chunks)} chunks. Example snippet:\n")
print(chunks[0].page_content[:200])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Retrieved 5 chunks. Example snippet:

a negative. the buy now and pay later option is a great for consumers, however the consumer is exposed for the payments ... if the vendor does not deliver the good and the consumer is still responsibl


In [8]:
# 3) Build prompt
prompt = build_prompt(chunks, QUESTION, PROMPT_TPL)

# 4) Call OpenRouter
answer = generate_openrouter(
    prompt=prompt,
    base_url=OPENROUTER_URL,
    api_key=OPENROUTER_KEY,
    model_name=OPENROUTER_MODEL
)

print("=== FINAL ANSWER ===\n")
print(answer)


=== FINAL ANSWER ===

Customers are unhappy with Buy Now, Pay Later (BNPL) services for several reasons highlighted in the complaints:  
1. **Responsibility for undelivered goods**: Consumers remain liable for payments even if vendors fail to deliver purchased items.  
2. **Punitive late fees**: High fees are criticized for worsening financial struggles, making it harder to recover from debt.  
3. **Suspected fee exploitation**: Customers accuse companies of intentionally stalling resolutions to prolong fee collection, especially in a challenging economic climate.  
4. **Unfair treatment despite good history**: Even those who consistently paid on time report feeling targeted by opaque or "malicious" practices.  
5. **Unexpected interest charges**: Delays in resolving issues (e.g., disputes) lead to unanticipated interest, with customers blaming the company’s slow response times.  

These grievances reflect broader frustrations about transparency, fairness, and corporate accountability 

In [9]:
print("\n=== SOURCES ===\n")
for doc in chunks:
    m = doc.metadata
    snippet = doc.page_content.replace("\n", " ")[:200]
    print(f"- [id={m['complaint_id']}, product={m['product']}] “{snippet}…”")



=== SOURCES ===

- [id=15378, product=Payday loan, title loan, personal loan, or advance loan] “a negative. the buy now and pay later option is a great for consumers, however the consumer is exposed for the payments ... if the vendor does not deliver the good and the consumer is still responsibl…”
- [id=89553, product=Credit card] “at a time that many of us are barely getting by i cant understand why they would take this hideous position with their own customer. when did we let late fees become whips that truly makes it impossib…”
- [id=94913, product=Payday loan, title loan, personal loan, or advance loan] “through situations in life, in which a company should care about the customers they serve and the outcome of their situation. now it just seems as if it s all being stalled so that the company can col…”
- [id=19313, product=Credit card] “unfair to the consumer. i have never been late on any payments and have always paid as agreed. please investigate their malicious practices. if i